In [10]:
"""
Fork of https://www.kaggle.com/antmarakis/bi-lstm-conv-layer?scriptVersionId=2789290
Just replaced the data with Preprocessed data
Public LB score 0.9833 => 0.9840

"""

import numpy as np
import pandas as pd
from keras.layers import Dense, Input, LSTM, Bidirectional, Conv1D, CuDNNLSTM
from keras.layers import Dropout, Embedding
from keras.preprocessing import text, sequence
from keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D
from keras.models import Model
from unidecode import unidecode
import re

train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [4]:
special_character_removal=re.compile(r'[^a-z\?\!\#\@\%\* ]',re.IGNORECASE)
def clean_text(x):
    x_ascii = unidecode(x)
    x_clean = special_character_removal.sub('',x_ascii)
    return x_clean

train['clean_text'] = train['comment_text'].apply(lambda x: clean_text(str(x)))
test['clean_text'] = test['comment_text'].apply(lambda x: clean_text(str(x)))

In [6]:
max_features=100000
maxlen=150
embed_size=300



train_y = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values
train_x = train['clean_text'].str.lower()

test_x = test['clean_text'].str.lower()

In [7]:
EMBEDDING_FILE="../data/fasttext/crawl-300d-2M.vec"

# Vectorize text + Prepare fasttext Embedding
tokenizer = text.Tokenizer(num_words=max_features, lower=True)
tokenizer.fit_on_texts(list(train_x))

train_x = tokenizer.texts_to_sequences(train_x)
test_x = tokenizer.texts_to_sequences(test_x)

train_x = sequence.pad_sequences(train_x, maxlen=maxlen)
test_x = sequence.pad_sequences(test_x, maxlen=maxlen)

embeddings_index = {}
with open(EMBEDDING_FILE, encoding='utf8') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

word_index = tokenizer.word_index
num_words = min(max_features, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embed_size))
for word, i in word_index.items():
    if i >= max_features:
        continue
    
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [19]:
# Build Model
inp = Input(shape=(maxlen,))

x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=True)(inp)
x = SpatialDropout1D(0.35)(x)

x = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x)
#x = Dropout(0.3)(x) #dropout doesnt improve
x = Conv1D(64, kernel_size=3, padding='valid', kernel_initializer='glorot_uniform')(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool, max_pool])

out = Dense(6, activation='sigmoid')(x)

model = Model(inp, out)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
# Prediction
batch_size = 512
epochs = 5

model.fit(train_x, train_y, batch_size=batch_size, epochs=epochs, verbose=1)

Epoch 1/5
159571/159571 [==============================] - 85s 532us/step - loss: 0.0725 - acc: 0.9757
Epoch 2/5
159571/159571 [==============================] - 83s 521us/step - loss: 0.0441 - acc: 0.9833
Epoch 3/5
159571/159571 [==============================] - 83s 522us/step - loss: 0.0393 - acc: 0.9847
Epoch 4/5
159571/159571 [==============================] - 83s 523us/step - loss: 0.0357 - acc: 0.9860
Epoch 5/5
159571/159571 [==============================] - 83s 523us/step - loss: 0.0326 - acc: 0.9870


In [21]:
predictions = model.predict(test_x, batch_size=batch_size, verbose=1)

153164/153164 [==============================] - 23s 147us/step


In [22]:
# Create submission file
output=pd.DataFrame(data=predictions, index=test["id"])
output.to_csv("./output/bilstm_conv_dropout.csv",header=["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
              ,index=True)